# Baseline 알고리즘 기반 추천시스템

## #01. 준비작업

### [1] 패키지 가져오기

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Intel SKlearn 하드웨어 가속 패치 설정
import sys
if sys.platform == 'win32':
    from sklearnex import patch_sklearn
    patch_sklearn()

from helper.util import *
from helper.plot import *
from helper.analysis import *
from helper.classification import *

from surprise import Reader, Dataset, BaselineOnly
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


### [2] 데이터 가져오기

In [2]:
origin = my_read_excel("https://data.hossam.kr/mldata/movie_ratings.xlsx", sheet_name='ratings', save=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None

데이터프레임 상위 5개 행
+----+----------+-----------+----------+-------------+
|    |   userId |   movieId |   rating |   timestamp |
|----+----------+-----------+----------+-------------|
|  0 |        1 |         1 |        4 | 9.64983e+08 |
|  1 |        1 |         3 |        4 | 9.64981e+08 |
|  2 |        1 |         6 |        4 | 9.64982e+08 |
|  3 |        1 |        47 |        5 | 9.64984e+08 |
|  4 |        1 |        50 |        5 | 9.64983e+08 |
+----+----------+-----------+----------+-------------+

데이터프레임 하위 5개 행
+--------+----------+-----------+----------+-------------+
|  

## #02. 데이터 전처리

### [1] Surprise 형식의 데이터로 변환

`사용자 번호, 아이템 번호, 평점` 구조의 데이터를 만족해야 한다.

In [24]:
df = origin.drop('timestamp', axis=1)

# 평점의 분포를 알려준다.
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(df, reader)
data

## #03. 추천 모형 구현

### [1] 기본 코드

#### (1) 훈련, 검증 데이터 분리

sklearn이 아닌 surprise 자체 함수 사용

In [29]:
train, test = train_test_split(data, test_size=0.2, random_state=1234)

#### (2) 추천 모형 학습 후 성능 평가

In [36]:
estimator = BaselineOnly()
estimator.fit(train)
pred = estimator.test(test)
pred[:5]

Estimating biases using als...


[Prediction(uid=603, iid=3996, r_ui=5.0, est=3.865470094018238, details={'was_impossible': False}),
 Prediction(uid=199, iid=2912, r_ui=4.0, est=3.5270186068257785, details={'was_impossible': False}),
 Prediction(uid=416, iid=2716, r_ui=2.0, est=3.2531312891488335, details={'was_impossible': False}),
 Prediction(uid=589, iid=150, r_ui=4.0, est=4.143871102075767, details={'was_impossible': False}),
 Prediction(uid=307, iid=6755, r_ui=4.0, est=2.6344308636371943, details={'was_impossible': False})]

> uid: 사용자 번호, iid: 아이템 번호, r_ui: 해당 사용자가 실제로 부여한 평점, est: 예측평점

#### (3) 성능평가

In [35]:
score = rmse(pred)

RMSE: 0.8715
